# 📘 Data Merging & Cleaning – Yousra Descriptive Stats Notebook

This notebook loads the original project datasets (sales, weather, kiwo event),  
cleans and standardizes the date format, merges them using a full outer join,  
and performs descriptive statistics, missing value inspection, and prepares the  
data for further feature engineering and modeling.

The goal is to:
1. Combine **all available data** on matching dates  
2. Perform a clear descriptive exploration  
3. Detect and handle missing values  
4. Produce a clean dataset ready for analysis  


In [7]:
import pandas as pd
import numpy as np

# Load data from your project folder
df_sales = pd.read_csv("data/kiwo.csv")
df_weather = pd.read_csv("data/wetter.csv")
df_umsatz = pd.read_csv("data/umsatzdaten_gekuerzt.csv")

df_sales.head(), df_weather.head(), df_umsatz.head()


(        Datum  KielerWoche
 0  2012-06-16            1
 1  2012-06-17            1
 2  2012-06-18            1
 3  2012-06-19            1
 4  2012-06-20            1,
         Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
 0  2012-01-01         8.0      9.8250                   14        58.0
 1  2012-01-02         7.0      7.4375                   12         NaN
 2  2012-01-03         8.0      5.5375                   18        63.0
 3  2012-01-04         4.0      5.6875                   19        80.0
 4  2012-01-05         6.0      5.3000                   23        80.0,
         Datum  Warengruppe      Umsatz
 0  2013-07-01            1  148.828353
 1  2013-07-02            1  159.793757
 2  2013-07-03            1  111.885594
 3  2013-07-04            1  168.864941
 4  2013-07-05            1  171.280754)

In [8]:
# Convert 'Datum' to proper datetime format and drop invalid rows
for df in (df_sales, df_weather, df_umsatz):
    df["Datum"] = pd.to_datetime(df["Datum"], errors="coerce").dt.normalize()
    df.dropna(subset=["Datum"], inplace=True)

    # Drop duplicate date columns if exist
    if "date" in df.columns:
        df.drop(columns=["date"], inplace=True)

df_sales.info(), df_weather.info(), df_umsatz.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Datum        72 non-null     datetime64[ns]
 1   KielerWoche  72 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2601 entries, 0 to 2600
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Datum                2601 non-null   datetime64[ns]
 1   Bewoelkung           2591 non-null   float64       
 2   Temperatur           2601 non-null   float64       
 3   Windgeschwindigkeit  2601 non-null   int64         
 4   Wettercode           1932 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 101.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10899 entries, 0 to

(None, None, None)

In [9]:
# Adding suffixes helps avoid confusion during merging

df_sales = df_sales.add_suffix("_sales")
df_sales.rename(columns={"Datum_sales": "Datum"}, inplace=True)

df_weather = df_weather.add_suffix("_weather")
df_weather.rename(columns={"Datum_weather": "Datum"}, inplace=True)

df_umsatz = df_umsatz.add_suffix("_umsatz")
df_umsatz.rename(columns={"Datum_umsatz": "Datum"}, inplace=True)


In [10]:
merged_df = (
    df_sales
    .merge(df_weather, on="Datum", how="outer")
    .merge(df_umsatz, on="Datum", how="outer")
)

merged_df.shape
merged_df.head()


,Datum,KielerWoche_sales,Bewoelkung_weather,Temperatur_weather,Windgeschwindigkeit_weather,Wettercode_weather,Warengruppe_umsatz,Umsatz_umsatz
0,2012-01-01,NaN,8.0,9.8250,14.0,58.0,NaN,NaN
1,2012-01-02,NaN,7.0,7.4375,12.0,NaN,NaN,NaN
2,2012-01-03,NaN,8.0,5.5375,18.0,63.0,NaN,NaN
3,2012-01-04,NaN,4.0,5.6875,19.0,80.0,NaN,NaN
4,2012-01-05,NaN,6.0,5.3000,23.0,80.0,NaN,NaN


In [11]:
# Numeric summary
merged_df.describe()

# For all columns (including categorical)
merged_df.describe(include="all")


,Datum,KielerWoche_sales,Bewoelkung_weather,Temperatur_weather,Windgeschwindigkeit_weather,Wettercode_weather,Warengruppe_umsatz,Umsatz_umsatz
count,11395,250.0,11259.000000,11314.000000,11314.000000,8621.000000,10899.000000,10899.000000
mean,2016-05-05 12:58:19.218955776,1.0,4.756906,11.850370,11.038183,36.477091,3.090834,206.663010
min,2012-01-01 00:00:00,1.0,0.000000,-10.250000,3.000000,0.000000,1.000000,7.051201
25%,2014-11-02 00:00:00,1.0,3.000000,6.187500,8.000000,10.000000,2.000000,97.532597
50%,2016-05-15 00:00:00,1.0,6.000000,11.375000,10.000000,25.000000,3.000000,163.299762
75%,2017-11-30 00:00:00,1.0,7.000000,17.625000,13.000000,61.000000,4.000000,280.813505
max,2019-08-01 00:00:00,1.0,8.000000,32.671428,35.000000,95.000000,6.000000,1879.461831
std,NaN,0.0,2.633733,7.154170,4.136318,27.155086,1.490105,142.811267


In [12]:
missing_count = merged_df.isna().sum()
missing_percent = (merged_df.isna().sum() / len(merged_df)) * 100

missing_df = pd.DataFrame({
    "Missing Count": missing_count,
    "Missing %": missing_percent
})

missing_df


,Missing Count,Missing %
Datum,0,0.000000
KielerWoche_sales,11145,97.806055
Bewoelkung_weather,136,1.193506
Temperatur_weather,81,0.710838
Windgeschwindigkeit_weather,81,0.710838
Wettercode_weather,2774,24.344011
Warengruppe_umsatz,496,4.352786
Umsatz_umsatz,496,4.352786


### ✔️ Missing Value Strategy

**Umsatz (Sales):**
- Missing values mean the day has **no sales** or is **outside the bakery event period**.
- These rows should be **removed**, not imputed.

**Weather Variables:**
- Weather data may be missing because not all dates have weather records.
- Fill missing values using interpolation (numerical)  
  and assign -1 for Wettercode to indicate “unknown weather”.

**Kiwo Event Flag:**
- If missing → fill with 0 (day outside event).


In [13]:
clean_df = merged_df.copy()

# Remove rows where sales are missing
clean_df = clean_df.dropna(subset=["Umsatz_umsatz"])

# Fill missing Wettercode with category -1
if "Wettercode_weather" in clean_df.columns:
    clean_df["Wettercode_weather"] = clean_df["Wettercode_weather"].fillna(-1)

# Interpolate numerical weather values
for col in clean_df.columns:
    if ("_weather" in col) and (clean_df[col].dtype in ["float64", "int64"]):
        clean_df[col] = clean_df[col].interpolate()

clean_df.isna().sum()


Datum                              0
KielerWoche_sales              10676
Bewoelkung_weather                 0
Temperatur_weather                 0
Windgeschwindigkeit_weather        0
Wettercode_weather                 0
Warengruppe_umsatz                 0
Umsatz_umsatz                      0
dtype: int64

In [14]:
clean_df["KielerWoche_sales"] = clean_df["KielerWoche_sales"].fillna(0)


In [15]:
clean_df.isna().sum()


Datum                          0
KielerWoche_sales              0
Bewoelkung_weather             0
Temperatur_weather             0
Windgeschwindigkeit_weather    0
Wettercode_weather             0
Warengruppe_umsatz             0
Umsatz_umsatz                  0
dtype: int64